수면장애(불면증)

문제제기

 잠은 우리의 3대 욕구중 가장 강한 욕구
 현대인들의 적 불면증

In [ ]:
# 라이브러리 로드
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/국민건강보험공단_수면장애(G47) 및 비기질성수면장애(F51) 진료인원_20221231.csv", encoding='cp949')
df

주상병코드      주상병명   진료년도       시도   시군구      연령 진료인원(명)
0       F51  비기질성수면장애  2010년    서울특별시   종로구  10대 미만     NaN
1       F51  비기질성수면장애  2010년    서울특별시   종로구     10대      12
2       F51  비기질성수면장애  2010년    서울특별시   종로구     20대      67
3       F51  비기질성수면장애  2010년    서울특별시   종로구     30대     108
4       F51  비기질성수면장애  2010년    서울특별시   종로구     40대     138
...     ...       ...    ...      ...   ...     ...     ...
57553   G47      수면장애  2022년  제주특별자치도  서귀포시     40대     570
57554   G47      수면장애  2022년  제주특별자치도  서귀포시     50대     665
57555   G47      수면장애  2022년  제주특별자치도  서귀포시     60대     701
57556   G47      수면장애  2022년  제주특별자치도  서귀포시     70대     500
57557   G47      수면장애  2022년  제주특별자치도  서귀포시  80대 이상     362

[57558 rows x 7 columns]

In [ ]:

df = pd.read_csv('/content/drive/MyDrive/국민건강보험공단_수면장애(G47) 및 비기질성수면장애(F51) 진료인원_20221231.csv', encoding='cp949', skiprows=[1])

df['진료인원(명)'] = df['진료인원(명)'].str.replace(',', '')
df['진료인원(명)'] = df['진료인원(명)'].fillna(0).astype(int)

grouped_data = df.groupby('진료년도')['진료인원(명)'].sum().reset_index()

plt.figure(figsize=(12, 6))
plt.plot(grouped_data['진료년도'], grouped_data['진료인원(명)'], marker='o')
plt.xlabel('진료년도')
plt.ylabel('총 진료인원(명)')
plt.title('진료년도에 따른 총 진료인원 변화')
plt.grid(True)

plt.xticks(rotation=45)
plt.show()



/usr/local/lib/python3.10/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 52509 (\N{HANGUL SYLLABLE CONG}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.10/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 51652 (\N{HANGUL SYLLABLE JIN}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.10/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 47308 (\N{HANGUL SYLLABLE RYO}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.10/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 51064 (\N{HANGUL SYLLABLE IN}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.10/dist-packages/IPython/core/pylabtools.py:151: UserWarning: Glyph 50896 (\N{HANGUL SYLLABLE WEON}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/usr/local/lib/python3.10/dist-packages

연도에 따른 불면증 환자가 지속적으로 증가하고 있음을 알 수 있다.

# 데이터로드

https://www.kaggle.com/datasets/uom190346a/sleep-health-and-lifestyle-dataset?select=Sleep_health_and_lifestyle_dataset.csv (데이터 출처)


Person ID: An identifier for each individual.
Gender: The gender of the person (Male/Female).
Age: The age of the person in years.
Occupation: The occupation or profession of the person.
Sleep Duration (hours): The number of hours the person sleeps per day.(수면시간)
Quality of Sleep (scale: 1-10): A subjective rating of the quality of sleep, ranging from 1 to 10. (수면의 질 1-10까지 평가하여 나눈것)
Physical Activity Level (minutes/day): The number of minutes the person engages in physical activity daily. (하루당 신체활동 시간(분단위))
Stress Level (scale: 1-10): A subjective rating of the stress level experienced by the person, ranging from 1 to 10.(스트레스레벨을 1-10까지 평가하여 나눈것)
BMI Category: The BMI category of the person (e.g., Underweight, Normal, Overweight). (비만 정도)
Blood Pressure (systolic/diastolic): The blood pressure measurement of the person, indicated as systolic pressure over diastolic pressure. (혈압)
Heart Rate (bpm): The resting heart rate of the person in beats per minute.(휴식 심박수)
Daily Steps: The number of steps the person takes per day. (하루의 걸음)
Sleep Disorder: The presence or absence of a sleep disorder in the person (None, Insomnia, Sleep Apnea). (수면장애 여부)

In [ ]:
dfs = pd.read_csv("/content/drive/MyDrive/Sleep_health_and_lifestyle_dataset.csv", encoding='cp949')
dfs['Sleep Disorder'] = dfs['Sleep Disorder'].apply(lambda x: 'Have' if x not in [None, 'None'] else x)

dfs = dfs.drop(['Person ID', 'Occupation','Gender'], axis=1)
dfs

Age  Sleep Duration  Quality of Sleep  Physical Activity Level  \
0     27            6.10                 6                       42   
1     28            6.20                 6                       60   
2     28            6.20                 6                       60   
3     28            5.90                 4                       30   
4     28            5.90                 4                       30   
..   ...             ...               ...                      ...   
369   59            8.10                 9                       75   
370   59            8.00                 9                       75   
371   59            8.10                 9                       75   
372   59            8.10                 9                       75   
373   59            8.10                 9                       75   

     Stress Level BMI Category Blood Pressure  Heart Rate  Daily Steps  \
0               6   Overweight         126/83          77         4200   
1               8       Normal         125/80          75        10000   
2               8       Normal         125/80          75        10000   
3               8        Obese         140/90          85         3000   
4               8        Obese         140/90          85         3000   
..            ...          ...            ...         ...          ...   
369             3   Overweight         140/95          68         7000   
370             3   Overweight         140/95          68         7000   
371             3   Overweight         140/95          68         7000   
372             3   Overweight         140/95          68         7000   
373             3   Overweight         140/95          68         7000   

    Sleep Disorder  
0             None  
1             None  
2             None  
3             Have  
4             Have  
..             ...  
369           Have  
370           Have  
371           Have  
372           Have  
373           Have  

[374 rows x 10 columns]

In [ ]:
bmi_category_unique_values = dfs['BMI Category'].unique()
print(bmi_category_unique_values)

['Overweight' 'Normal' 'Obese' 'Normal Weight']


In [ ]:
#수치화를 위해 BMI Category 칼럼의 Normal와 Normal Weight를 1 Overweight를 3 , Obese 5 으로 변환
#Sleep Disorder은 0,1로변환
#혈압은 수축기 혈압 기준(높은 쪽) 정상 :~120, 주의혈압 121~130  고혈압 전: 130  고혈압 1기 :140~159  초고혈압:160~  1 2 3 4 5 로 변환
dfs['BMI Category'] = dfs['BMI Category'].replace({'Normal': 1, 'Normal Weight': 1, 'Overweight': 3, 'Obese': 5})
dfs['Sleep Disorder'] = dfs['Sleep Disorder'].replace({'None': 0, 'Have': 1})
dfs['Blood Pressure'] = dfs['Blood Pressure'].astype(str).str.split('/').str[0].astype(int)

dfs['Blood Pressure'] = pd.cut(dfs['Blood Pressure'], bins=[0, 120, 130, 140, 160, float('inf')], labels=[1, 2, 3, 4, 5])
dfs

Age  Sleep Duration  Quality of Sleep  Physical Activity Level  \
0     27            6.10                 6                       42   
1     28            6.20                 6                       60   
2     28            6.20                 6                       60   
3     28            5.90                 4                       30   
4     28            5.90                 4                       30   
..   ...             ...               ...                      ...   
369   59            8.10                 9                       75   
370   59            8.00                 9                       75   
371   59            8.10                 9                       75   
372   59            8.10                 9                       75   
373   59            8.10                 9                       75   

     Stress Level  BMI Category Blood Pressure  Heart Rate  Daily Steps  \
0               6             3              2          77         4200   
1               8             1              2          75        10000   
2               8             1              2          75        10000   
3               8             5              3          85         3000   
4               8             5              3          85         3000   
..            ...           ...            ...         ...          ...   
369             3             3              3          68         7000   
370             3             3              3          68         7000   
371             3             3              3          68         7000   
372             3             3              3          68         7000   
373             3             3              3          68         7000   

     Sleep Disorder  
0                 0  
1                 0  
2                 0  
3                 1  
4                 1  
..              ...  
369               1  
370               1  
371               1  
372               1  
373               1  

[374 rows x 10 columns]

# 결측치 확인

In [ ]:
dfs.isnull().sum()

Age                        0
Sleep Duration             0
Quality of Sleep           0
Physical Activity Level    0
Stress Level               0
BMI Category               0
Blood Pressure             0
Heart Rate                 0
Daily Steps                0
Sleep Disorder             0
dtype: int64

# 데이터 분석 및 요약

In [ ]:
pd.set_option('display.float_format', '{:.2f}'.format)
dfs.describe()

Age  Sleep Duration  Quality of Sleep  Physical Activity Level  \
count 374.00          374.00            374.00                   374.00   
mean   42.18            7.13              7.31                    59.17   
std     8.67            0.80              1.20                    20.83   
min    27.00            5.80              4.00                    30.00   
25%    35.25            6.40              6.00                    45.00   
50%    43.00            7.20              7.00                    60.00   
75%    50.00            7.80              8.00                    75.00   
max    59.00            8.50              9.00                    90.00   

       Stress Level  BMI Category  Heart Rate  Daily Steps  Sleep Disorder  
count        374.00        374.00      374.00       374.00          374.00  
mean           5.39          1.90       70.17      6816.84            0.41  
std            1.77          1.10        4.14      1617.92            0.49  
min            3.00          1.00       65.00      3000.00            0.00  
25%            4.00          1.00       68.00      5600.00            0.00  
50%            5.00          1.00       70.00      7000.00            0.00  
75%            7.00          3.00       72.00      8000.00            1.00  
max            8.00          5.00       86.00     10000.00            1.00

In [ ]:
dfs['Sleep Disorder'].value_counts()

0    219
1    155
Name: Sleep Disorder, dtype: int64

Gender	Age	Sleep Duration	Quality of Sleep	Physical Activity Level	Stress Level	BMI Category	Blood Pressure	Heart Rate	Daily Steps를 독립변수 X값으로 두고 이에 따른 Sleep disorder의 여부를 Y값으로 두고 상관관계를 파악하고자함
로지스틱 회귀모형으로 보는 것이 좋겠다고 판단.


In [ ]:
X = dfs.drop(columns=['Sleep Disorder'])
# Y값을 11번째 열로 설정
Y = dfs['Sleep Disorder']

# 데이터를 학습용과 테스트용으로 나눔
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.7, test_size=0.3, random_state=1234, stratify=Y)

print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

# 로지스틱 회귀 모델 생성
model = LogisticRegression(max_iter=1000)

# 모델을 학습시킴
model_train=model.fit(X_train, Y_train)
# 테스트 데이터로 모델을 평가
Y_pred = model.predict(X_test)

accuracy = accuracy_score(Y_test, Y_pred)
print("Accuracy: ", accuracy)

(261, 9) (113, 9) (261,) (113,)
Accuracy:  0.8849557522123894


In [ ]:
X.head()

Age  Sleep Duration  Quality of Sleep  Physical Activity Level  \
0   27            6.10                 6                       42   
1   28            6.20                 6                       60   
2   28            6.20                 6                       60   
3   28            5.90                 4                       30   
4   28            5.90                 4                       30   

   Stress Level  BMI Category Blood Pressure  Heart Rate  Daily Steps  
0             6             3              2          77         4200  
1             8             1              2          75        10000  
2             8             1              2          75        10000  
3             8             5              3          85         3000  
4             8             5              3          85         3000

In [ ]:
Y.tail()

369    1
370    1
371    1
372    1
373    1
Name: Sleep Disorder, dtype: int64

In [ ]:
# Y_pred를 threshold를 기준으로 0과 1로 나누어주는 함수 생성
def cut_off(y,threshold):
    Y = y.copy() # copy함수를 사용하여 이전의 y값이 변화지 않게 함
    Y[Y>threshold]=1
    Y[Y<=threshold]=0
    return(Y.astype(int))

In [ ]:
Y_predict_t = cut_off(Y_pred,0.5)

In [ ]:
accuracy = accuracy_score(Y_test, Y_predict_t)
precision = precision_score(Y_test, Y_predict_t)
recall = recall_score(Y_test, Y_predict_t)
f1 = f1_score(Y_test, Y_predict_t)
print('Accuracy : {0:.3f}'.format(accuracy))
print('Precision : {0:.3f}'.format(precision))
print('Recall : {0:.3f}'.format(recall))
print('F1 : {0:.3f}'.format(f1))

Accuracy : 0.885
Precision : 0.870
Recall : 0.851
F1 : 0.860
